In [19]:
import geopandas as gpd
import pandas as pd

pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

cities = gpd.read_feather(
    "/Users/test/Documents/code/IL2/factory_loc_service/examples/closest_cities.feather"
)

uinput_industry = "oil_and_gas_ext"
uinput_spec_num = {"Машинист": 200, "Оператор, аппаратчик": 300}
city_names = cities["region_city"].values

In [20]:
print(uinput_industry, uinput_spec_num, city_names)

oil_and_gas_ext {'Машинист': 200, 'Оператор, аппаратчик': 300} ['Ростовская область, Аксай' 'Ростовская область, Белая Калитва'
 'Ростовская область, Зверево' 'Ростовская область, Константиновск'
 'Ростовская область, Красный Сулин' 'Ростовская область, Новочеркасск'
 'Ростовская область, Новошахтинск' 'Ростовская область, Семикаракорск'
 'Ростовская область, Шахты']


In [21]:
ontology = pd.read_pickle("new_ontology.pkl")

grouped_grads = pd.read_pickle("grouped_grads.pkl")

cv = pd.read_parquet("../api/app/data/cv.gzip").rename(
    columns={"hh_name": "speciality"}
)
YEAR = 2021
cv = cv[cv["year"] == YEAR]

In [22]:
mask = grouped_grads.reset_index(drop=False).loc[:, "cluster_center"].isin(city_names)
grouped_grads = (
    grouped_grads.reset_index(drop=False)
    .loc[mask, :]
    .set_index(["cluster_center", "region_city", "type", "edu_group_code"])
)

mask = cv["region_city"].isin(city_names)
cv = cv[mask]

In [23]:
# ontology = pd.read_csv(
#     "/Users/test/Documents/code/IL2/industrial-location/api/app/data/ontology.csv",
#     index_col=0,
# )

# ontology.loc[ontology["industry_code"] == "mining_oil_gas", "industry_code"] = (
#     "oil_and_gas_ext"
# )
# ontology.loc[ontology["industry_code"] == "mmining_coal_ores", "industry_code"] = (
#     "coal_and_metal_ext"
# )
# ontology.loc[ontology["industry_code"] == "mech_engineering", "industry_code"] = (
#     "machinery"
# )
# ontology.loc[ontology["industry_code"] == "shipbuilding", "industry_code"] = (
#     "shipbuilding"
# )
# ontology.loc[ontology["industry_code"] == "aircraft_engineering", "industry_code"] = (
#     "aircraft_and_space"
# )
# ontology.loc[ontology["industry_code"] == "non_ferrous_metallurgy", "industry_code"] = (
#     "nonferrous_metallurgy"
# )
# ontology.loc[ontology["industry_code"] == "ferrous_metallurgy", "industry_code"] = (
#     "ferrous_metallurgy"
# )
# ontology.loc[ontology["industry_code"] == "chemical", "industry_code"] = "chemicals"
# ontology.loc[ontology["industry_code"] == "pharma", "industry_code"] = "pharmacy"
# ontology.loc[ontology["industry_code"] == "electronics", "industry_code"] = (
#     "electronics"
# )

In [24]:
# ontology["industry_code"].unique()

In [25]:
# ontology.to_pickle("new_ontology.pkl")

In [26]:
cv = cv.merge(ontology[["speciality", "edu_group_code"]], on="speciality")
cv["type"] = "CV"
grouped_cv = (
    cv.groupby(["cluster_center", "region_city", "type", "edu_group_code"])["id_cv"]
    .count()
    .to_frame()
)
cv.head(3)

,id_cv,speciality,year,region_city,cluster_center,edu_group_code,type
0,148ef4b0-54bc-11eb-904a-736ab11edb0c,Инженер-конструктор,2021,"Ростовская область, Новочеркасск","Ростовская область, Новочеркасск",15,CV
1,148ef4b0-54bc-11eb-904a-736ab11edb0c,Инженер-конструктор,2021,"Ростовская область, Новочеркасск","Ростовская область, Новочеркасск",24,CV
2,148ef4b0-54bc-11eb-904a-736ab11edb0c,Инженер-конструктор,2021,"Ростовская область, Новочеркасск","Ростовская область, Новочеркасск",11,CV


In [27]:
# grads.dropna(subset="edu_group", inplace=True)
# grouped_grads = (
#     grads.groupby(["cluster_center", "region_city", "type", "edu_group_code"])[
#         "graduates_per_year_forecast"
#     ]
#     .sum()
#     .to_frame()
# )
# grouped_grads = grouped_grads.join(grouped_cv, how="outer")
# grouped_grads["city_capacity_grads_and_cv_sum"] = grouped_grads[
#     "graduates_per_year_forecast"
# ].fillna(0) + grouped_grads["id_cv"].fillna(0)
# grouped_grads.to_pickle("grouped_grads.pkl")

In [28]:
# --- START ---
"""
Эта штука завязана на входных профессиях и индустрии,
поэтому нужно каждый раз пересчитывать -- мб можно оптимизировать и пересчитывать только часть
но имхо оно и так быстро
"""

uinput_spec_num_2 = dict()
competitor_industries = list()
competitor_fatories = list()


for k in uinput_spec_num.keys():
    uinput_spec_num_2[k] = (
        ontology.loc[ontology["speciality"] == k, "edu_group_id"]
        .drop_duplicates()
        .values.tolist()
    )
    competitor_industries += (
        ontology.loc[ontology["speciality"] == k, "industry_code"]
        .drop_duplicates()
        .tolist()
    )
# --- START ---

competitor_industries = set(competitor_industries)
competitor_industries.remove(uinput_industry)


grad_col = []
fact_col = []

for col in cities.columns:
    if "factor" in col and uinput_industry in col:
        fact_col.append(col)

    if "factor" in col and any(industry in col for industry in competitor_industries):
        competitor_fatories.append(col)

    if "grad" in col and uinput_industry in col:
        grad_col.append(col)

cities["competitors_factories_num"] = cities[competitor_fatories].sum(axis=1)

print(competitor_industries)
print(grad_col, fact_col, competitor_fatories)

{'pharmacy', 'nonferrous_metallurgy', 'ferrous_metallurgy', 'machinery', 'shipbuilding', 'electronics', 'mining_coal_ores', 'chemicals'}
['graduates_oil_and_gas_ext'] ['factories_oil_and_gas_ext'] ['factories_machinery', 'factories_shipbuilding', 'factories_pharmacy', 'factories_chemicals', 'factories_nonferrous_metallurgy', 'factories_ferrous_metallurgy', 'factories_electronics']


In [29]:
grouped_grads = (
    grouped_grads.reset_index(drop=False)
    .groupby(["cluster_center", "type"])[["city_capacity_grads_and_cv_sum", "id_cv"]]
    .sum()
    .join(
        cities[
            ["region_city", "population", "num_in_migration"]
            + grad_col
            + fact_col
            + ["factories_total"]
            + ["competitors_factories_num"]
        ]
        .rename(columns={"region_city": "cluster_center"})
        .set_index("cluster_center"),
        how="left",
    )
)

grouped_grads[
    [
        "num_in_migration",
        "graduates_oil_and_gas_ext",
        "factories_oil_and_gas_ext",
        "factories_total",
        "city_capacity_grads_and_cv_sum",
        "population",
        "competitors_factories_num",
    ]
] = (
    grouped_grads[
        [
            "num_in_migration",
            "graduates_oil_and_gas_ext",
            "factories_oil_and_gas_ext",
            "factories_total",
            "city_capacity_grads_and_cv_sum",
            "population",
            "competitors_factories_num",
        ]
    ]
    .round(0)
    .fillna(0)
    .astype(int)
)

grouped_grads["working_population"] = (
    (grouped_grads["population"] * 0.65).round(0).astype(int)
)

In [30]:
"""
(!) Сейчас есть всё кроме непосредственной оценки (!)
А ну и задаваемое кол-во специалистов пока не учитывается
Но это следствие из отсутствия реализации расчета оценки

Эту таблицу практически в таком виде и планирую отдавать

Из того что обсуждали с СА:
условно выпускников взвесить по кол-ву предприятий тк их число ту мач большое
"""

'\n(!) Сейчас есть всё кроме непосредственной оценки (!)\nА ну и задаваемое кол-во специалистов пока не учитывается\nНо это следствие из отсутствия реализации расчета оценки\n\nЭту таблицу практически в таком виде и планирую отдавать\n\nИз того что обсуждали с СА:\nусловно выпускников взвесить по кол-ву предприятий тк их число ту мач большое\n'

In [31]:
grouped_grads

city_capacity_grads_and_cv_sum   id_cv  population  num_in_migration  graduates_oil_and_gas_ext  factories_oil_and_gas_ext  factories_total  competitors_factories_num  working_population
cluster_center                     type                                                                                                                                                                                            
Ростовская область, Аксай          CV                               260   260.0       48714                40                          0                          0                7                          7               31664
                                   СПО                               94     0.0       48714                40                          0                          0                7                          7               31664
Ростовская область, Белая Калитва  CV                               578   578.0       39457                 3                          0                          0                2                          2               25647
                                   СПО                              166     0.0       39457                 3                          0                          0                2                          2               25647
Ростовская область, Зверево        CV                               143   143.0       18718                10                          0                          0                2                          1               12167
                                   СПО                               47     0.0       18718                10                          0                          0                2                          1               12167
Ростовская область, Константиновск CV                               320   320.0       16910                 1                         17                          0                0                          0               10992
                                   СПО                               88     0.0       16910                 1                         17                          0                0                          0               10992
Ростовская область, Красный Сулин  CV                               463   463.0       35347                 5                          0                          0                0                          0               22976
                                   СПО                               88     0.0       35347                 5                          0                          0                0                          0               22976
Ростовская область, Новочеркасск   CV                              1809  1809.0      160782               105                       3036                          0               14                         14              104508
                                   ВПО                             7118     0.0      160782               105                       3036                          0               14                         14              104508
                                   СПО                              707     0.0      160782               105                       3036                          0               14                         14              104508
Ростовская область, Новошахтинск   CV                               275   275.0      101708                33                          0                          0                1                          1               66110
                                   СПО                              267     0.0      101708                33                          0                          0                1                          1               66110
Ростовская область, Семикаракорск  CV                               487   487.0       21488                 1                          0                          0  

In [32]:
""" ------------------------- FINISH HERE -------------------------------- """

' ------------------------- FINISH HERE -------------------------------- '